In [41]:
import optuna 
import xgboost as xgb
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import sklearn
from sklearn.metrics import mean_squared_error
import operator
from random import seed
from random import random

In [42]:
best = pd.read_csv('..\\kaggle_data\\best.csv')
working_df = best.copy(deep=True)
# seed random number generator
seed(1)

In [43]:
for index, row in working_df.iterrows():
    do_work = random()
    if do_work > 0.5:
        direction = random()
        if direction > 0.5:
            working_df.at[index,'target'] = row.target + random()/16
        else:
            working_df.at[index,'target'] = row.target - random()/16

In [44]:
distance_mse_to_best = mean_squared_error(working_df, best)
distance_rmse_to_best = math.sqrt(distance_mse_to_best)
print(f'Distance to Best {distance_rmse_to_best}')
print(working_df.head())

Distance to Best 0.01808495959016943
   id    target
0   0  7.932313
1   2  7.901713
2   6  7.936487
3   7  8.285162
4  10  8.106503


In [45]:
from datetime import datetime
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
print(working_df.head())
working_df.to_csv(f'..\\kaggle_data\\{dt_string}_submission.csv', index=False)

   id    target
0   0  7.932313
1   2  7.901713
2   6  7.936487
3   7  8.285162
4  10  8.106503
